# Problématique métier

Le but est de prédire la condition opérationel du "point d'eau".

Les status possibles sont les suivants : 
* functional
* functional needs repair
* non functional

# Préparation des données

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
#pd.set_option('max_columns', None)

In [13]:
raw_data = pd.read_csv("input/features.csv", ",")

D:\ESGI\5AL\ML\ML-TP\venv\lib\site-packages\IPython\core\interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
# Regarder dans la base
raw_data.head() # afficher les 5 premières lignes

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [8]:
# Vérification si chaque lignes à bien un id
len(raw_data["id"].unique())

59400

In [15]:
print(raw_data.shape)
raw_data.info()

(59400, 40)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int

## Recherche des valeurs manquantes

In [19]:
# nombre de valeurs manquantes
print(raw_data.isnull().sum().sort_values(ascending=False))

scheme_name              28166
scheme_management         3877
installer                 3655
funder                    3635
public_meeting            3334
permit                    3056
subvillage                 371
management_group             0
waterpoint_type              0
extraction_type              0
extraction_type_group        0
source_class                 0
extraction_type_class        0
management                   0
source_type                  0
source                       0
payment_type                 0
water_quality                0
construction_year            0
quality_group                0
quantity                     0
quantity_group               0
payment                      0
id                           0
amount_tsh                   0
recorded_by                  0
population                   0
ward                         0
lga                          0
district_code                0
region_code                  0
region                       0
basin   

Après analyse, nous allons enlever le scheme_name

In [20]:
# Proportion de valeurs manquantes
print(raw_data.isnull().mean().sort_values(ascending=False))

scheme_name              0.474175
scheme_management        0.065269
installer                0.061532
funder                   0.061195
public_meeting           0.056128
permit                   0.051448
subvillage               0.006246
management_group         0.000000
waterpoint_type          0.000000
extraction_type          0.000000
extraction_type_group    0.000000
source_class             0.000000
extraction_type_class    0.000000
management               0.000000
source_type              0.000000
source                   0.000000
payment_type             0.000000
water_quality            0.000000
construction_year        0.000000
quality_group            0.000000
quantity                 0.000000
quantity_group           0.000000
payment                  0.000000
id                       0.000000
amount_tsh               0.000000
recorded_by              0.000000
population               0.000000
ward                     0.000000
lga                      0.000000
district_code 

## Préparation des données

In [21]:
string_list = ["funder", "installer", "wpt_name", "basin", "subvillage", "region", "lga", "ward", "recorded_by", "scheme_management", "scheme_name", "extraction_type", "extraction_type_group", "extraction_type_class", "management", "management_group", "payment", "payment_type", "water_quality", "quality_group", "quantity", "quantity_group", "source", "source_type", "source_class", "waterpoint_type", "waterpoint_type_group"]

In [22]:
raw_data_conv = raw_data.copy()
#a mettre avant
raw_data_conv[string_list] = raw_data_conv[string_list].astype(str)
raw_data_conv[string_list] = raw_data_conv[string_list].astype("string")
raw_data_conv[string_list] = raw_data_conv[string_list].replace({pd.NA: np.nan})
raw_data_conv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 59400 non-null  string 
 4   gps_height             59400 non-null  int64  
 5   installer              59400 non-null  string 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  string 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  string 
 11  subvillage             59400 non-null  string 
 12  region                 59400 non-null  string 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

Convervion des dates

In [25]:
raw_data_conv['date_recorded'] =  pd.to_datetime(raw_data_conv['date_recorded'], format='%Y-%m-%d')

In [26]:
raw_data_conv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     59400 non-null  int64         
 1   amount_tsh             59400 non-null  float64       
 2   date_recorded          59400 non-null  datetime64[ns]
 3   funder                 59400 non-null  string        
 4   gps_height             59400 non-null  int64         
 5   installer              59400 non-null  string        
 6   longitude              59400 non-null  float64       
 7   latitude               59400 non-null  float64       
 8   wpt_name               59400 non-null  string        
 9   num_private            59400 non-null  int64         
 10  basin                  59400 non-null  string        
 11  subvillage             59400 non-null  string        
 12  region                 59400 non-null  string        
 13  r

Convertion des colonnes "object" restantes en ???

In [29]:
# récupérer les colonnes à convertir
objectcols = raw_data_conv.select_dtypes(include='object').columns
print(objectcols)

Index(['public_meeting', 'permit'], dtype='object')


In [30]:
# Convertion des données en boolean
raw_data_conv[objectcols] = raw_data_conv[objectcols].astype(bool)
raw_data_conv[objectcols] = raw_data_conv[objectcols].astype("bool")
raw_data_conv[objectcols] = raw_data_conv[objectcols].replace({pd.NA: np.nan})

In [31]:
raw_data_conv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     59400 non-null  int64         
 1   amount_tsh             59400 non-null  float64       
 2   date_recorded          59400 non-null  datetime64[ns]
 3   funder                 59400 non-null  string        
 4   gps_height             59400 non-null  int64         
 5   installer              59400 non-null  string        
 6   longitude              59400 non-null  float64       
 7   latitude               59400 non-null  float64       
 8   wpt_name               59400 non-null  string        
 9   num_private            59400 non-null  int64         
 10  basin                  59400 non-null  string        
 11  subvillage             59400 non-null  string        
 12  region                 59400 non-null  string        
 13  r

## Préparation de la target

In [33]:
target = pd.read_csv("input/target.csv", delimiter=",")
y = target.drop("id", axis=1)
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status_group  59400 non-null  object
dtypes: object(1)
memory usage: 464.2+ KB


In [34]:
# Convertion du "status_group" en string
col = "status_group"
y[col] = y[col].astype(str)
y[col] = y[col].astype("string")
y[col] = y[col].replace({pd.NA: np.nan})
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status_group  59400 non-null  string
dtypes: string(1)
memory usage: 464.2 KB


# Imputation & Sélection des variables explicatives

In [ ]:
raw_data_conv = raw_data_conv.dropna(axis=1, how='all')
raw_data_conv.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Classe_energetique,Security
0,04/02/2015 17:51,23.18,27.2720,426.0,721.25,0.004793,A,oui
1,04/02/2015 17:51,23.15,27.2675,429.5,714.00,0.004783,A,oui
2,04/02/2015 17:53,23.15,27.2450,426.0,713.50,0.004779,A,oui
3,04/02/2015 17:54,23.15,27.2000,426.0,708.25,0.004772,A,oui
4,04/02/2015 17:55,23.10,27.2000,426.0,704.50,0.004757,A,oui


In [ ]:
from datetime import datetime
raw_data_conv['date'] = pd.to_datetime(raw_data_conv['date'])
raw_data_conv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17895 entries, 0 to 17894
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                17895 non-null  datetime64[ns]
 1   Temperature         17871 non-null  float64       
 2   Humidity            17895 non-null  float64       
 3   Light               17764 non-null  float64       
 4   CO2                 17830 non-null  float64       
 5   HumidityRatio       17872 non-null  float64       
 6   Classe_energetique  17895 non-null  string        
 7   Security            17895 non-null  string        
dtypes: datetime64[ns](1), float64(5), string(2)
memory usage: 1.1 MB


In [ ]:
print(raw_data_conv['Security'].value_counts())
raw_data_conv['Security'] = raw_data_conv['Security'].replace("nan", "non")
raw_data_conv["Security"].value_counts()

non    12684
oui     5160
nan       51
Name: Security, dtype: Int64


non    12735
oui     5160
Name: Security, dtype: Int64

In [ ]:
print(raw_data_conv['Classe_energetique'].value_counts())


C      7974
D      4710
B      3311
A      1871
nan      29
Name: Classe_energetique, dtype: Int64


In [ ]:
#raw_data_conv['Security'].value_counts()
# récupérer les colonnes à convertir
floatcols = raw_data_conv.select_dtypes(include='float64').columns
print(floatcols)

Index(['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio'], dtype='object')


In [ ]:
data_median =  raw_data_conv[floatcols].fillna(raw_data_conv[floatcols].median())
data_moyenne = raw_data_conv[floatcols].fillna(raw_data_conv[floatcols].mean())
data_mode = raw_data_conv[floatcols].fillna(raw_data_conv[floatcols].mode())

In [ ]:
print(" les stats descriptives après imputation par la médiane :", data_median.describe())
print(" les stats descriptives après imputation par la moyenne :", data_moyenne.describe())
print(" les stats descriptives après imputation par le mode :", data_mode.describe())
print(" les stats descriptives avant imputation:", raw_data_conv.describe())

 /nles stats descriptives après imputation par la médiane :         Temperature      Humidity         Light           CO2  HumidityRatio
count  17895.000000  17895.000000  17895.000000  17895.000000   17895.000000
mean      20.826040     27.998747    121.453183    686.791801       0.004258
std        1.035847      5.169896    202.207255    313.505558       0.000784
min       19.000000     16.745000      0.000000    412.750000       0.002674
25%       20.100000     24.890000      0.000000    458.750000       0.003767
50%       20.600000     27.793333      0.000000    565.000000       0.004343
75%       21.500000     31.856667    217.916667    792.000000       0.004860
max       24.390000     39.500000   1581.000000   2076.500000       0.006476
 /nles stats descriptives après imputation par la moyenne :         Temperature      Humidity         Light           CO2  HumidityRatio
count  17895.000000  17895.000000  17895.000000  17895.000000   17895.000000
mean      20.826343     27.998747

In [ ]:
raw_data_conv.isnull().mean().sort_values(ascending=False)

Light                 0.007320
CO2                   0.003632
Temperature           0.001341
HumidityRatio         0.001285
date                  0.000000
Humidity              0.000000
Classe_energetique    0.000000
Security              0.000000
dtype: float64

In [ ]:
print(raw_data_conv.shape)
# 1 pour supprimer les colonnes contenant des valeurs manquantes
data_supp_col = raw_data_conv.dropna(axis=1)

print(data_supp_col.shape)

(17895, 8)
(17895, 4)


In [ ]:
data_supp_col.head()

,date,Humidity,Classe_energetique,Security
0,2015-04-02 17:51:00,27.2720,A,oui
1,2015-04-02 17:51:00,27.2675,A,oui
2,2015-04-02 17:53:00,27.2450,A,oui
3,2015-04-02 17:54:00,27.2000,A,oui
4,2015-04-02 17:55:00,27.2000,A,oui
